#Importo librerie

In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=d42583b1391ef7e4d3af09b72c273f02f50f2097d84c6bade1256f7953262431
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=21604289a8ce1377f3d8e7a357a8ac3effce85d5c24fccc214a2ddc0ea635eba
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import pandas as pd 
import numpy as np 
import itertools

from time import time

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, r2_score, precision_recall_fscore_support
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.utils import shuffle

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

#import smogn

from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Dropout, Conv1D, Flatten, MaxPooling1D, AveragePooling1D, SimpleRNN, LSTM
from keras.regularizers import l2, l1, l1_l2
from keras.constraints import max_norm
from keras.callbacks import EarlyStopping, History
from keras import optimizers

from kerastuner import HyperModel
from kerastuner.tuners import Hyperband

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


#Definizione funzioni

In [ ]:
# grafico matrice di confusione

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# stampa tempo trascorso

def print_exec_time(start):
  print("Addestramento completato in %f secondi" % (time()-start))

In [ ]:
# funzione per creare un sotto-df con le classi target bilanciate, in modo che anche le classi della variabile specificata nel parametro col siano egualmente rappresentate in ciascuna classe target

def undersample(df, label, col=None):
  hits = df[df[label] == 1]
  non_hits = df[df[label] == 0]

  if(col != None):

    non_hits_sampled_array = []

    for a in non_hits[col].unique():
      n = hits[hits[col] == a].id.count()
      non_hits_sampled_array.append(non_hits[non_hits[col] == a].sample(n))

    non_hits_sampled = pd.concat(non_hits_sampled_array)

  else:
    non_hits_sampled = df[df[label] == 0].sample(hits.shape[0])

  return pd.concat([hits,non_hits_sampled])

In [ ]:
def over_under_balancing(X, Y, oversample_ratio, oversample_algorithm):
  '''
  oversample_algorithm --> [SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN]
  '''
  over = oversample_algorithm(sampling_strategy=oversample_ratio) # --> genero nuovi esempi nella classe in minoranza (hit) in modo da portarla al (es.) 50% della classe maggioritaria (non-hit)
  under = RandomUnderSampler(sampling_strategy=1) # --> rimuovo casualmente esempi dalla classe maggioritaria (non-hit) fino a portarla al 100% della classe minoritaria (hit)
  steps = [('o', over),('u', under)]
  pipeline = Pipeline(steps=steps)

  X, Y = pipeline.fit_resample(X, Y)
  
  return X, Y

In [ ]:
# new_features_params
def insert_new_features_params(row, new_features_params):
    if(row.features == 'standard features'):
        return None
    else:
        return new_features_params

In [ ]:
# selezionare il numero di componenti principali per LDA

def select_n_components(var_ratio, goal_var: float) -> int:
    # Set initial variance explained so far
    total_variance = 0.0
    
    # Set initial number of features
    n_components = 0
    
    # For the explained variance of each feature:
    for explained_variance in var_ratio:
        
        # Add the explained variance to the total
        total_variance += explained_variance
        
        # Add one to the number of components
        n_components += 1
        
        # If we reach our goal level of explained variance
        if total_variance >= goal_var:
            # End the loop
            break
            
    # Return the number of components
    return n_components

#Importo Dataset

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
drive.CreateFile({'id':'1-0o81KniM9hNtC8zqBaYyQWAmGdTYSS5'}).GetContentFile('dataset_final_4.0.csv')
df = pd.read_csv("dataset_final_4.0.csv").drop('Unnamed: 0',axis=1)

In [ ]:
#

#DL

In [ ]:
category = 'deep learning'
problem = 'classification'
oversample_algorithm = SMOTE
class_balancing = oversample_algorithm.__name__ + ' + random undersampling'

earlyStopping = EarlyStopping(monitor='loss', min_delta=0.001, patience=10, restore_best_weights=True)

result_df_array = []

x = 0 # --> parametro da variare per considerare le new_features calcolate sui diversi intorni di anni

# # # FEATURES SELECT # # #

std_features_list = ['valence','acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','explicit','key_0','key_1','key_2','key_3','key_4','key_5','key_6','key_7','key_8','key_9','key_10','key_11','mode'] 

if(x == 'all'):
  new_features = ['valence_new','acousticness_new','danceability_new','duration_ms_new','energy_new','instrumentalness_new','liveness_new','loudness_new','speechiness_new','tempo_new','explicit_new','key_new','mode_new']
  new_features_list = []

  for i in range(4):
    for feat in new_features:
      new_features_list.append(feat + '_' + str(i))
else:
  new_features_list = ['valence_new_'+str(x),'acousticness_new_'+str(x),'danceability_new_'+str(x),'duration_ms_new_'+str(x),'energy_new_'+str(x),'instrumentalness_new_'+str(x),'liveness_new_'+str(x),'loudness_new_'+str(x),'speechiness_new_'+str(x),'tempo_new_'+str(x),'explicit_new_'+str(x),'key_new_'+str(x),'mode_new_'+str(x)]

extra_features = ['month','year','cos(month)','sin(month)','season_1','season_2','season_3','season_4', 'past_pop_n_hit', 'past_pop_n_weeks']
targets_list = ['hit', 'weeks_enc']

features_select = {'standard features': std_features_list+extra_features, 'standard + new features': std_features_list+new_features_list+extra_features, 'new features': new_features_list+extra_features}

for current_features in features_select.keys():

    if(current_features != 'standard features'):
      new_features_params = x
    else:
      new_features_params = None

    # # # YEAR RANGE SELECT # # #

    year_range_select = [(1960,2020), (1960,1969), (1970,1979), (1980,1989), (1990,1999), (2000,2009), (2010,2020)]

    for year_range in year_range_select:

      year_start = year_range[0]
      year_end = year_range[1]

      # seleziono sotto_df
      mask_1 = df.year_YYYY >= year_start
      mask_2 = df.year_YYYY <= year_end
      sub_df = df[mask_1]
      sub_df = sub_df[mask_2]

      # regolo dimensione test set in base a numero di anni considerato
      if((year_end - year_start) > 10):
          test_size = 0.2
          val_size = int(0.05 * sub_df.shape[0])
      else:
          test_size = 0.3
          val_size = int(0.08 * sub_df.shape[0])
      
      # creo validation set
      sub_df = shuffle(sub_df)
      val_set = sub_df.iloc[:val_size].copy()
      sub_df = sub_df.iloc[val_size:].copy()

      # seleziono features correnti
      features = features_select[current_features]

      # creo array numpy
      X_val = val_set[features].drop(['month'], axis=1).values
      Y_val = val_set['hit'].values
      
      X = sub_df[features].drop(['month'], axis=1).values
      Y = sub_df['hit'].values
      
      # creo set addestramento e test
      X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)
      
      # bilancio train set
      X_train, Y_train = over_under_balancing(X_train, Y_train, oversample_ratio=0.6, oversample_algorithm=oversample_algorithm)

      # bilancio test set
      X_test, Y_test = RandomUnderSampler(sampling_strategy=1).fit_resample(X_test, Y_test)

      # bilancio val set
      X_val, Y_val = RandomUnderSampler(sampling_strategy=1).fit_resample(X_val, Y_val)


      # ------------------------------------------- #

      # MODELLI

      # --- model_0 --- #

      model_0 = Sequential()
      # hidden layer
      model_0.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
      model_0.add(Dropout(0.75))
      # output layer
      model_0.add(Dense(1,activation='sigmoid'))

      # --- model_1 --- #

      model_1 = Sequential()
      # hidden layer
      model_1.add(Dense(1024, input_dim=X_train.shape[1], activation='relu'))
      model_1.add(Dropout(0.75))
      # hidden layer
      model_1.add(Dense(254, activation='relu'))
      model_1.add(Dropout(0.65))
      # hidden layer
      model_1.add(Dense(128, activation='relu'))
      model_1.add(Dropout(0.65))
      # hidden layer
      model_1.add(Dense(64, activation='relu'))
      model_1.add(Dropout(0.6))
      # output layer
      model_1.add(Dense(1,activation='sigmoid'))

      models = [model_0, model_1]

      for i, model in enumerate(models):

        adam = optimizers.Adam()  # learning_rate=learning_rates[i]

        model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
        print('\n')
        model.summary()

        time_0 = time()
        model.fit(X_train,Y_train, epochs=100, validation_data=(X_val, Y_val), batch_size=254, callbacks=[earlyStopping])
        print_exec_time(time_0)

        # METRICHE

        # --- metriche test set --- #

        # loss e accuracy
        loss, accuracy = model.evaluate(X_test,Y_test)

        # eseguo predizioni su test set
        Y_pred = (model.predict(X_test) > 0.5).astype("int32")

        # matrice di confusione
        cm = confusion_matrix(Y_test,Y_pred)

        # precision, recall, fscore
        precision, recall, fscore, support = precision_recall_fscore_support(Y_test, Y_pred)

        # --- metriche test set --- #

        # loss e accuracy
        loss_train, accuracy_train = model.evaluate(X_train, Y_train)

        # eseguo predizioni su test set
        Y_pred_train = (model.predict(X_train) > 0.5).astype("int32")

        # matrice di confusione
        cm_train = confusion_matrix(Y_train,Y_pred_train)

        # precision, recall, fscore
        precision_train, recall_train, fscore_train, support = precision_recall_fscore_support(Y_train, Y_pred_train)

        # imposto a None le metriche della regressione
        MSE = None
        r2 = None
        MSE_train = None
        r2_train = None

        if(i == 1):
          parameters = '1 hidden layer (64)'
        else:
          parameters = '4 hidden layers (1024, 254, 128, 64)'

        tot_time = time() - time_0

        result_df = pd.DataFrame(
                [['MLP',
                parameters,
                class_balancing,
                problem,
                accuracy,
                loss,
                cm,
                precision[0],
                precision[1],
                recall[0],
                recall[1],
                fscore[0],
                fscore[1],
                accuracy_train,
                loss_train,
                cm_train,
                precision_train[0],
                precision_train[1],
                recall_train[0],
                recall_train[1],
                fscore_train[0],
                fscore_train[1],
                MSE,
                r2,
                MSE_train,
                r2_train,
                tot_time]],
          columns=['algorithm',
                  'parameters',
                  'class_balancing',
                  'problem',
                  'test_accuracy',
                  'test_log_loss',
                  'test_confusion_matrix',
                  'test_precision_0',
                  'test_precision_1', 
                  'test_recall_0',
                  'test_recall_1',
                  'test_fscore_0',
                  'test_fscore_1',
                  'train_accuracy',
                  'train_log_loss',
                  'train_confusion_matrix',
                  'train_precision_0',
                  'train_precision_1',
                  'train_recall_0',
                  'train_recall_1',
                  'train_fscore_0',
                  'train_fscore_1',
                  'test_MSE',
                  'test_r2',
                  'train_MSE',
                  'train_r2',
                  'tot_time'])
        

        # inserisco campi mancanti

        # year_range
        year_range_array = [year_range for i in range(result_df.shape[0])]
        result_df.insert(0, 'year_range', year_range_array)

        # features
        features_array = [current_features for i in range(result_df.shape[0])]
        result_df.insert(0, 'features', features_array)

        # new_features_params
        new_features_params_array = [new_features_params for i in range(result_df.shape[0])]
        result_df.insert(0, 'new_features_params', new_features_params_array)

        # category
        category_array = ['deep learning' for i in range(result_df.shape[0])]
        result_df.insert(0, 'category', category_array)
        
        result_df_array.append(result_df)

df_tot = pd.concat(result_df_array)



/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                2176      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 2,241
Trainable params: 2,241
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
358/358 [==============================] - 5s 4ms/step - loss: 0.6981 - accuracy: 0.5375 - val_loss: 0.6375 - val_accuracy: 0.6393
Epoch 2/100
358/358 [==============================] - 1s 2ms/step - loss: 0.6420 - accuracy: 0.6209 - val_loss: 0.6208 - val_accuracy: 0.6479
Epoch 3/100
358/358 [==============================] - 1s 2ms/step - loss: 0.6238 - accuracy: 0.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 64)                2176      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 2,241
Trainable params: 2,241
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
53/53 [==============================] - 1s 5ms/step - loss: 0.7347 - accuracy: 0.5076 - val_loss: 0.6794 - val_accuracy: 0.5656
Epoch 2/100
53/53 [==============================] - 0s 3ms/step - loss: 0.6833 - accuracy: 0.5548 - val_loss: 0.6714 - val_accuracy: 0.6011
Epoch 3/100
53/53 [==============================] - 0s 3ms/step - loss: 0.6656 - accuracy: 0.5860

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_index

...



Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_126 (Dense)            (None, 64)                1472      
_________________________________________________________________
dropout_90 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_127 (Dense)            (None, 1)                 65        
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
51/51 [==============================] - 1s 6ms/step - loss: 0.7394 - accuracy: 0.5068 - val_loss: 0.6837 - val_accuracy: 0.5789
Epoch 2/100
51/51 [==============================] - 0s 3ms/step - loss: 0.7001 - accuracy: 0.5148 - val_loss: 0.6742 - val_accuracy: 0.5822
Epoch 3/100
51/51 [==============================] - 0s 3ms/step - loss: 0.6802 - accuracy: 0.551

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_index



Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_133 (Dense)            (None, 64)                1472      
_________________________________________________________________
dropout_95 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_134 (Dense)            (None, 1)                 65        
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
47/47 [==============================] - 1s 6ms/step - loss: 0.7581 - accuracy: 0.5064 - val_loss: 0.6842 - val_accuracy: 0.5718
Epoch 2/100
47/47 [==============================] - 0s 3ms/step - loss: 0.7055 - accuracy: 0.5221 - val_loss: 0.6797 - val_accuracy: 0.5743
Epoch 3/100
47/47 [==============================] - 0s 3ms/step - loss: 0.6886 - accuracy: 0.527

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_index



Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_140 (Dense)            (None, 64)                1472      
_________________________________________________________________
dropout_100 (Dropout)        (None, 64)                0         
_________________________________________________________________
dense_141 (Dense)            (None, 1)                 65        
Total params: 1,537
Trainable params: 1,537
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
53/53 [==============================] - 1s 5ms/step - loss: 0.7360 - accuracy: 0.5109 - val_loss: 0.6834 - val_accuracy: 0.5783
Epoch 2/100
53/53 [==============================] - 0s 3ms/step - loss: 0.6974 - accuracy: 0.5251 - val_loss: 0.6771 - val_accuracy: 0.5841
Epoch 3/100
53/53 [==============================] - 0s 3ms/step - loss: 0.6824 - accuracy: 0.560

In [ ]:
df_tot.sort_values('test_accuracy', ascending=False).head(50)

,category,new_features_params,features,year_range,algorithm,parameters,class_balancing,problem,test_accuracy,test_log_loss,test_confusion_matrix,test_precision_0,test_precision_1,test_recall_0,test_recall_1,test_fscore_0,test_fscore_1,train_accuracy,train_log_loss,train_confusion_matrix,train_precision_0,train_precision_1,train_recall_0,train_recall_1,train_fscore_0,train_fscore_1,test_MSE,test_r2,train_MSE,train_r2,tot_time
0,deep learning,None,standard features,"(2010, 2020)",MLP,"4 hidden layers (1024, 254, 128, 64)",SMOTE + random undersampling,classification,0.744669,0.544433,"[[1331, 498], [436, 1393]]",0.753254,0.736647,0.727720,0.761618,0.740267,0.748925,0.751079,0.529659,"[[4759, 1958], [1386, 5331]]",0.774451,0.731376,0.708501,0.793658,0.740009,0.761245,None,None,None,None,18.446626
0,deep learning,None,standard features,"(2010, 2020)",MLP,1 hidden layer (64),SMOTE + random undersampling,classification,0.741115,0.543428,"[[1416, 413], [534, 1295]]",0.726154,0.758197,0.774194,0.708037,0.749405,0.732259,0.781003,0.466912,"[[5224, 1493], [1449, 5268]]",0.782856,0.779175,0.777728,0.784279,0.780284,0.781718,None,None,None,None,22.878101
0,deep learning,0,standard + new features,"(1960, 2020)",MLP,"4 hidden layers (1024, 254, 128, 64)",SMOTE + random undersampling,classification,0.721527,0.550151,"[[3286, 1192], [1302, 3176]]",0.716216,0.727106,0.733810,0.709245,0.724906,0.718065,0.758411,0.507119,"[[33144, 12332], [9641, 35835]]",0.774664,0.743974,0.728824,0.787998,0.751045,0.765354,None,None,None,None,70.805876
0,deep learning,0,standard + new features,"(2010, 2020)",MLP,"4 hidden layers (1024, 254, 128, 64)",SMOTE + random undersampling,classification,0.721519,0.570028,"[[1339, 478], [534, 1283]]",0.714896,0.728563,0.736929,0.706109,0.725745,0.717160,0.745024,0.536067,"[[5021, 1711], [1722, 5010]]",0.744624,0.745425,0.745841,0.744207,0.745232,0.744815,None,None,None,None,19.500836
0,deep learning,0,standard + new features,"(1960, 2020)",MLP,1 hidden layer (64),SMOTE + random undersampling,classification,0.721304,0.554819,"[[3351, 1127], [1369, 3109]]",0.709958,0.733947,0.748325,0.694283,0.728637,0.713564,0.775431,0.475788,"[[34280, 11196], [9229, 36247]]",0.787883,0.764012,0.753804,0.797058,0.770467,0.780185,None,None,None,None,120.908746
0,deep learning,0,standard + new features,"(2010, 2020)",MLP,1 hidden layer (64),SMOTE + random undersampling,classification,0.719318,0.570563,"[[1369, 448], [572, 1245]]",0.705307,0.735381,0.753440,0.685195,0.728579,0.709402,0.744950,0.533170,"[[5173, 1559], [1875, 4857]]",0.733967,0.757014,0.768419,0.721480,0.750798,0.738820,None,None,None,None,14.197771
0,deep learning,0,new features,"(2000, 2009)",MLP,1 hidden layer (64),SMOTE + random undersampling,classification,0.718933,0.562629,"[[1142, 226], [543, 825]]",0.677745,0.784967,0.834795,0.603070,0.748117,0.682100,0.719761,0.560488,"[[4841, 1095], [2232, 3704]]",0.684434,0.771827,0.815532,0.623989,0.744254,0.690079,None,None,None,None,17.218617
0,deep learning,None,standard features,"(1980, 1989)",MLP,"4 hidden layers (1024, 254, 128, 64)",SMOTE + random undersampling,classification,0.718902,0.561131,"[[566, 254], [207, 613]]",0.732212,0.707036,0.690244,0.747561,0.710609,0.726734,0.767074,0.492531,"[[4315, 2069], [905, 5479]]",0.826628,0.725888,0.675909,0.858239,0.743709,0.786535,None,None,None,None,17.120228
0,deep learning,0,new features,"(2000, 2009)",MLP,"4 hidden layers (1024, 254, 128, 64)",SMOTE + random undersampling,classification,0.717105,0.572505,"[[1050, 318], [456, 912]]",0.697211,0.741463,0.767544,0.666667,0.730689,0.702079,0.718076,0.574651,"[[4375, 1561], [1786, 4150]]",0.710112,0.726668,0.737028,0.699124,0.723320,0.712630,None,None,None,None,18.387799
0,deep learning,0,new features,"(2010, 2020)",MLP,1 hidden layer (64),SMOTE + random undersampling,classification,0.717087,0.578727,"[[1341, 444], [566, 1219]]",0.703199,0.733013,0.751261,0.682913,0.726436,0.707077,0.722555,0.571853,"[[4926, 1760], [1950, 4736]]",0.7164

# Export

In [ ]:
from google.colab import  drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_tot.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/results_DL_4.1_'+str(x)+'.csv')

Mounted at /content/drive
